# Séléction de modèle 

In [1]:
import matplotlib
%matplotlib nbagg
from matplotlib import pyplot

In [2]:
from statiskit import (linalg,
                       core,
                       pgm)

In [3]:
import math
import os

In [4]:
for lasso in ['AUTRE', 'LINEAIRE']:
    os.environ['LASSO'] = lasso
    mod = !jupyter nbconvert --ExecutePreprocessor.timeout=3600 --to notebook --execute misc/LASSO.ipynb --output misc/LASSO.ipynb
graph0 = pgm.read_gml(os.path.join('results', 'graph.gml'))
data = core.read_csv(os.path.join('results', 'data.csv'))
data

,V0,V1,V2,V3,V4,V5,V6,V7,V8,V9
0,$0.20$,$-0.48$,$2.25$,$1.59$,$-0.35$,$0.75$,$-1.48$,$0.61$,$3.28$,$-2.61$
1,$1.06$,$-0.12$,$-0.68$,$0.22$,$1.75$,$1.79$,$-1.82$,$0.46$,$1.25$,$0.65$
2,$-1.33$,$-0.50$,$-0.18$,$0.89$,$-1.70$,$0.13$,$0.07$,$1.20$,$0.44$,$-2.82$
3,$1.51$,$-0.56$,$0.64$,$2.65$,$0.11$,$1.30$,$-2.94$,$1.14$,$1.18$,$-0.34$
4,$0.77$,$-0.05$,$-1.42$,$-1.55$,$-1.01$,$0.58$,$-0.75$,$1.34$,$0.10$,$-2.74$
5,$0.02$,$-0.50$,$0.43$,$-1.71$,$-0.43$,$1.05$,$-0.30$,$0.79$,$2.08$,$-0.23$
6,$-0.66$,$0.09$,$-1.51$,$0.08$,$-0.09$,$0.50$,$0.07$,$1.68$,$-0.68$,$-2.83$
7,$1.45$,$-0.59$,$-0.95$,$-1.89$,$-0.85$,$0.83$,$2.22$,$0.64$,$0.12$,$-1.97$
8,$0.24$,$-0.10$,$-0.48$,$-1.72$,$-0.19$,$0.41$,$0.80$,$0.64$,$0.10$,$0.25$
9,$-1.07$,$-0.51$,$-0.45$,$0.64$,$0.58$,$1.56$,$-1.71$,$0.45$,$2.32$,$0.34$


In [5]:
graphs = []
for filepath in os.listdir('.'):
    if filepath.startswith('lasso_'):
        adjmat = linalg.read_csv(filepath)
        graphs.append(pgm.UndirectedGraph(adjmat))
graphs = sorted(graphs, key = lambda graph: graph.nb_edges)
graphs

[]

In [9]:
LLHs = []
BICs = []
AICs = []

%time
for graph in graphs:
    try:
        mle = pgm.graphical_gaussian_estimation(algo='scd',
                                                data=data,
                                                graph=graph)
        dist = mle.estimated.copy()

        LLHs.append(2 * dist.loglikelihood(data))
        
    except Exception as e:
        print e
        LLHs.append(float("nan"))
        BICs.append(LLHs[-1])
        AICs.append(LLHs[-1])
    else:
        BICs.append(LLHs[-1] - dist.nb_parameters * math.log(len(data.events)))
        AICs.append(LLHs[-1] - dist.nb_parameters * 2)

#fig = pyplot.figure()
#axes = fig.add_subplot(111)
#axes.plot(LLHs)
#axes.plot(BICs)
#axes.plot(AICs)


CPU times: user 18 µs, sys: 2 µs, total: 20 µs
Wall time: 49.8 µs


[]

In [7]:
os.environ['CRITER'] = "bic"
criter = os.environ.get('CRITER')
criter

'bic'

In [8]:
if criter == "bic":
    model = BICs.index(max(BICs))
else:
    model = AICs.index(max(AICs))
model

ValueError: max() arg is an empty sequence

In [ ]:
import math
with open(os.path.join('results', 'model.csv'), 'a') as filehandler:
    filehandler.write(','.join([str(criter), str(model)]) + '\n')